In [18]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [19]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [20]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
# latitude =
# longitude =

earliest_bike_datetime = "2023-08-05 12:56:53 UTC"
last_bike_datetime = "2025-01-06 15:06:11 UTC"

today = datetime.datetime.now()

2025-01-07 12:29:18,796 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 12:29:18,803 INFO: Initializing external client
2025-01-07 12:29:18,804 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 12:29:20,041 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [21]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [22]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [23]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Loop through the commits and convert the bike data into a dataframe

In [24]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [25]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


Processing commit:  68290eccd75b7d4e47320383a23d86e7a43eff1c  -  2025-01-06 23:49:02+00:00
Processing commit:  a7bc71c83506577ff7dc6e6687e852d36651c6c8  -  2025-01-06 22:49:02+00:00
Processing commit:  3397f6bd8f1013ee14a88f3cb6dc46b7c4706db0  -  2025-01-06 21:49:09+00:00
Processing commit:  133f27c2e3b1237915f63add44b13ef61619910b  -  2025-01-06 20:49:03+00:00
Processing commit:  97141c31253eb495340fd2b1fa675c75fbfadf8e  -  2025-01-06 19:49:05+00:00
Processing commit:  d86a5ba24238c2a6d82943004d7998317a6158c1  -  2025-01-06 18:49:48+00:00
Processing commit:  e794fc5ce20ae14539f97f31a3315af922959f42  -  2025-01-06 17:49:18+00:00
Processing commit:  42967854469793e1d901cb67f10c74a619a510af  -  2025-01-06 16:50:06+00:00
Processing commit:  c307dd3308878520b876dde02b330ff3271e624e  -  2025-01-06 15:49:08+00:00
breaking at:  2025-01-06 15:06:11+00:00
{'CLARENDON ROW': [[1, datetime.datetime(2025, 1, 6, 23, 49, 2, tzinfo=<git.objects.util.tzoffset object at 0x000001A4367010F0>)], [2, dateti

In [26]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station
        df_bike_today = pd.concat([df_bike_today, df])
df_bike_today.dropna(inplace=True)
df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
df_bike_today


,num_bikes_available,datetime,station
0,1.0,2025-01-06 23:49:02+00:00,CLARENDON ROW
1,2.0,2025-01-06 22:49:02+00:00,CLARENDON ROW
2,3.0,2025-01-06 21:49:09+00:00,CLARENDON ROW
3,4.0,2025-01-06 20:49:03+00:00,CLARENDON ROW
4,0.0,2025-01-06 19:49:05+00:00,CLARENDON ROW
...,...,...,...
4,4.0,2025-01-06 19:49:05+00:00,HANOVER QUAY EAST
5,3.0,2025-01-06 18:49:48+00:00,HANOVER QUAY EAST
6,1.0,2025-01-06 17:49:18+00:00,HANOVER QUAY EAST
7,8.0,2025-01-06 16:50:06+00:00,HANOVER QUAY EAST


## Fetch the weather data for the same time period

In [27]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-07 00:00:00+00:00,1.00,-4.701722,0.0,0.0,23.110207,27904.794922,0.0,dublin
1,2025-01-07 01:00:00+00:00,0.85,-4.501354,0.0,0.0,20.721119,27904.794922,0.0,dublin
2,2025-01-07 02:00:00+00:00,0.60,-4.605682,0.0,0.0,19.486609,27904.794922,0.0,dublin
3,2025-01-07 03:00:00+00:00,0.50,-4.742946,0.0,0.0,19.799999,27904.794922,0.0,dublin
4,2025-01-07 04:00:00+00:00,0.70,-4.512824,0.0,0.0,20.089161,27904.794922,0.0,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-01-16 19:00:00+00:00,5.45,2.606128,0.0,0.0,10.739832,29183.220703,0.0,dublin
236,2025-01-16 20:00:00+00:00,5.30,2.404336,0.0,0.0,10.883676,29183.220703,0.0,dublin
237,2025-01-16 21:00:00+00:00,5.20,2.268395,0.0,0.0,10.990322,29183.220703,0.0,dublin
238,2025-01-16 22:00:00+00:00,5.05,2.094080,0.0,0.0,10.948973,29183.220703,0.0,dublin


## Insert the bike and weather data into Hopsworks

In [28]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

Uploading Dataframe: 100.00% |██████████| Rows 1026/1026 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bike_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


In [29]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df)


Uploading Dataframe: 100.00% |██████████| Rows 240/240 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions


## Change the last_bike_datetime secret

In [30]:
"""
if not df_bike_today.empty:
    try:
        secrets.get_secret("time_secrets").delete()
    except:
        pass

    new_last_bike_datetime_raw = pd.Series.max(df_bike_today['datetime'])
    new_last_bike_datetime = new_last_bike_datetime_raw.strftime('%Y-%m-%d %H:%M:%S %Z')

    time_secrets_dict = {
        "earliest_bike_datetime": earliest_bike_datetime,
        "last_bike_datetime": new_last_bike_datetime
    }

    secrets.create_secret("time_secrets", json.dumps(time_secrets_dict))
""" 

'\nif not df_bike_today.empty:\n    try:\n        secrets.get_secret("time_secrets").delete()\n    except:\n        pass\n\n    new_last_bike_datetime_raw = pd.Series.max(df_bike_today[\'datetime\'])\n    new_last_bike_datetime = new_last_bike_datetime_raw.strftime(\'%Y-%m-%d %H:%M:%S %Z\')\n\n    time_secrets_dict = {\n        "earliest_bike_datetime": earliest_bike_datetime,\n        "last_bike_datetime": new_last_bike_datetime\n    }\n\n    secrets.create_secret("time_secrets", json.dumps(time_secrets_dict))\n'